看年份有重疊的企業 CSR 報告，對應的 CSR 分數差距之統計數據

In [ ]:
import pandas as pd

In [ ]:
hm_NASDAQ_divided = pd.read_csv('../Model/output_dataset/handmade_features_NASDAQ_divided.csv')
hm_NYSE_divided = pd.read_csv('../Model/output_dataset/handmade_features_NYSE_divided.csv')

In [ ]:
# drop those rows with NaN
hm_NASDAQ_divided_cover = hm_NASDAQ_divided.dropna()[['exchange', 'ticker', 'year1', 'year2']]
hm_NYSE_divided_cover = hm_NYSE_divided.dropna()[['exchange', 'ticker', 'year1', 'year2']]

In [ ]:
hm_NASDAQ_divided_cover.sort_values(by=['exchange', 'ticker', 'year1', 'year2'], inplace=True)
hm_NYSE_divided_cover.sort_values(by=['exchange', 'ticker', 'year1', 'year2'], inplace=True)
# hm_NYSE_divided_cover.head(50)

In [ ]:
# drop those ticker only exist once
hm_NASDAQ_divided_cover_duplicate = hm_NASDAQ_divided_cover.groupby('ticker').filter(lambda x: len(x) > 1)
hm_NYSE_divided_cover_duplicate = hm_NYSE_divided_cover.groupby('ticker').filter(lambda x: len(x) > 1)
hm_NASDAQ_divided_cover_duplicate['year1'] = hm_NASDAQ_divided_cover_duplicate['year1'].astype(int)
hm_NYSE_divided_cover_duplicate['year1'] = hm_NYSE_divided_cover_duplicate['year1'].astype(int)

hm_NASDAQ_divided_cover_duplicate['prev_year'] = hm_NASDAQ_divided_cover_duplicate.groupby('ticker')['year1'].shift(1)
hm_NASDAQ_divided_cover_duplicate['year_diff'] = hm_NASDAQ_divided_cover_duplicate['year1'] - hm_NASDAQ_divided_cover_duplicate['prev_year']

hm_NYSE_divided_cover_duplicate['prev_year'] = hm_NYSE_divided_cover_duplicate.groupby('ticker')['year1'].shift(1)
hm_NYSE_divided_cover_duplicate['year_diff'] = hm_NYSE_divided_cover_duplicate['year1'] - hm_NYSE_divided_cover_duplicate['prev_year']

covered_NASDAQ = hm_NASDAQ_divided_cover_duplicate[hm_NASDAQ_divided_cover_duplicate['year_diff'] == 1]
covered_NASDAQ = covered_NASDAQ.drop(columns=['prev_year', 'year_diff'])

covered_NYSE = hm_NYSE_divided_cover_duplicate[hm_NYSE_divided_cover_duplicate['year_diff'] == 1]
covered_NYSE = covered_NYSE.drop(columns=['prev_year', 'year_diff'])

In [ ]:
covered_report = pd.concat([covered_NASDAQ, covered_NYSE])
covered_report.to_csv('../Model/output_dataset/covered_report.csv', index=False)

# Here

In [101]:
covered_report = pd.read_csv('../output_dataset/covered_report.csv')
covered_report.head(5)

,exchange,ticker,year1,year2
0,NASDAQ,AAON,2019,2020
1,NASDAQ,AAON,2020,2021
2,NASDAQ,AAON,2021,2022
3,NASDAQ,ADSK,2011,2012
4,NASDAQ,ADSK,2012,2013


In [102]:
common_score = pd.read_csv('../CSR_score/common_score_ESG.csv')
common_score.head(5)

,ticker,2005,2006,2007,2008,2009,2010,2011,2012,2013,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,PMT,0.000000,0.000000,14.460174,28.895211,27.795259,-3.226296,-8.438004,-5.548695,-6.044652,...,-15.010161,-19.205710,-17.039996,-18.401621,-14.321301,-18.761270,-25.068604,-23.502445,10.680420,0.0
1,ACVA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.787916,28.256582,0.0
2,TRP,1.973488,-5.840225,5.171804,24.878843,28.903738,28.397471,32.707430,24.143747,24.732660,...,8.441327,-2.133232,-13.172975,-11.473292,-15.849688,-14.023199,-18.216672,-24.164780,51.588543,0.0
3,VRTX,8.013624,31.797662,7.561376,-1.085975,-8.569791,-9.995450,-7.010220,-11.440721,-13.915639,...,-5.861065,0.139335,6.616973,25.312459,14.515943,-1.044804,8.244195,6.610504,0.000000,0.0
4,VTR,0.000000,0.000000,3.601887,3.202949,4.292841,-1.426620,-1.760391,9.865475,23.884559,...,25.346508,34.884566,28.779756,26.241391,24.608606,23.180766,21.341284,23.251231,79.003113,0.0


In [103]:
import pandas as pd

# 確保 ticker 格式一致（去除空格 & 轉大寫）
covered_report['ticker'] = covered_report['ticker'].str.strip().str.upper()
common_score['ticker'] = common_score['ticker'].str.strip().str.upper()

# 找出 missing_tickers
missing_tickers = set(covered_report['ticker'].unique()) - set(common_score['ticker'].unique())
print("\n🚨 無法在 common_score 中找到的 ticker:", missing_tickers)

# 移除 `missing_tickers`，只保留 common_score 有的 ticker
filtered_covered_report = covered_report[~covered_report['ticker'].isin(missing_tickers)].copy()
filtered_covered_report


🚨 無法在 common_score 中找到的 ticker: {'LND', 'AMD', 'CZR', 'VRTU', 'CVLT', 'KLIC', 'ISRG', 'OTEX', 'BRKS', 'HSIC', 'SCHN', 'LNDC', 'XLNX', 'MSFT'}


,exchange,ticker,year1,year2
0,NASDAQ,AAON,2019,2020
1,NASDAQ,AAON,2020,2021
2,NASDAQ,AAON,2021,2022
3,NASDAQ,ADSK,2011,2012
4,NASDAQ,ADSK,2012,2013
5,NASDAQ,ADSK,2013,2014
6,NASDAQ,ADSK,2014,2015
7,NASDAQ,ALKS,2019,2020
12,NASDAQ,AMZN,2019,2020
13,NASDAQ,AMZN,2020,2021


In [104]:
# 轉換 common_score 為 long format
common_score_long = common_score.melt(id_vars=['ticker'], var_name='year', value_name='GWscore')
common_score_long['year'] = common_score_long['year'].astype(int)  # 確保 year 是 int

# 合併 year1 的 GWscore
filtered_covered_report = filtered_covered_report.merge(
    common_score_long, left_on=['ticker', 'year1'], right_on=['ticker', 'year'], how='left'
)
filtered_covered_report.rename(columns={'GWscore': 'GWscore_year1'}, inplace=True)
filtered_covered_report.drop(columns=['year'], inplace=True)  # 刪除額外的 year 欄位

# 合併 year2 的 GWscore
filtered_covered_report = filtered_covered_report.merge(
    common_score_long, left_on=['ticker', 'year2'], right_on=['ticker', 'year'], how='left'
)
filtered_covered_report.rename(columns={'GWscore': 'GWscore_year2'}, inplace=True)
filtered_covered_report.drop(columns=['year'], inplace=True)  # 刪除額外的 year 欄位

# 計算 GWscore_diff
filtered_covered_report.loc[:, 'GWscore_diff'] = (
    filtered_covered_report['GWscore_year2'] - filtered_covered_report['GWscore_year1']
).abs()

filtered_covered_report

,exchange,ticker,year1,year2,GWscore_year1,GWscore_year2,GWscore_diff
0,NASDAQ,AAON,2019,2020,3.828476,6.211471,2.382995
1,NASDAQ,AAON,2020,2021,6.211471,3.948939,2.262532
2,NASDAQ,AAON,2021,2022,3.948939,8.713777,4.764838
3,NASDAQ,ADSK,2011,2012,12.434045,13.616089,1.182044
4,NASDAQ,ADSK,2012,2013,13.616089,18.260704,4.644616
5,NASDAQ,ADSK,2013,2014,18.260704,21.404091,3.143387
6,NASDAQ,ADSK,2014,2015,21.404091,18.537454,2.866637
7,NASDAQ,ALKS,2019,2020,11.420792,8.355700,3.065092
8,NASDAQ,AMZN,2019,2020,35.116077,33.472847,1.643230
9,NASDAQ,AMZN,2020,2021,33.472847,25.080838,8.392009


In [105]:
filtered_covered_report['GWscore_diff'].describe()

count    48.000000
mean      5.756160
std       5.381539
min       0.180186
25%       2.151686
50%       3.581547
75%       7.830243
max      23.094275
Name: GWscore_diff, dtype: float64

In [106]:
filtered_covered_report.to_csv('../output_dataset/covered_report_score.csv', index=False)

# Describe of all the greenwashing score

In [12]:
import pandas as pd

In [13]:
common_score = pd.read_csv('../CSR_score/common_score_ESG.csv')

In [14]:
common_score.head(5)

,ticker,2005,2006,2007,2008,2009,2010,2011,2012,2013,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,PMT,0.000000,0.000000,-14.460174,-28.895211,-27.795259,3.226296,8.438004,5.548695,6.044652,...,15.010161,19.205710,17.039996,18.401621,14.321301,18.761270,25.068604,23.502445,-10.680420,0.0
1,ACVA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.787916,-28.256582,0.0
2,TRP,-1.973488,5.840225,-5.171804,-24.878843,-28.903738,-28.397471,-32.707430,-24.143747,-24.732660,...,-8.441327,2.133232,13.172975,11.473292,15.849688,14.023199,18.216672,24.164780,-51.588543,0.0
3,VRTX,-8.013624,-31.797662,-7.561376,1.085975,8.569791,9.995450,7.010220,11.440721,13.915639,...,5.861065,-0.139335,-6.616973,-25.312459,-14.515943,1.044804,-8.244195,-6.610504,0.000000,0.0
4,VTR,0.000000,0.000000,-3.601887,-3.202949,-4.292841,1.426620,1.760391,-9.865475,-23.884559,...,-25.346508,-34.884566,-28.779756,-26.241391,-24.608606,-23.180766,-21.341284,-23.251231,-79.003113,0.0


In [15]:
common_score_abs = common_score.set_index('ticker').abs()
common_score_abs.head(5)

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
ticker,,,,,,,,,,,,,,,,,,,,
PMT,0.000000,0.000000,14.460174,28.895211,27.795259,3.226296,8.438004,5.548695,6.044652,13.201703,15.010161,19.205710,17.039996,18.401621,14.321301,18.761270,25.068604,23.502445,10.680420,0.0
ACVA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.787916,28.256582,0.0
TRP,1.973488,5.840225,5.171804,24.878843,28.903738,28.397471,32.707430,24.143747,24.732660,18.111797,8.441327,2.133232,13.172975,11.473292,15.849688,14.023199,18.216672,24.164780,51.588543,0.0
VRTX,8.013624,31.797662,7.561376,1.085975,8.569791,9.995450,7.010220,11.440721,13.915639,5.374071,5.861065,0.139335,6.616973,25.312459,14.515943,1.044804,8.244195,6.610504,0.000000,0.0
VTR,0.000000,0.000000,3.601887,3.202949,4.292841,1.426620,1.760391,9.865475,23.884559,28.567191,25.346508,34.884566,28.779756,26.241391,24.608606,23.180766,21.341284,23.251231,79.003113,0.0


In [16]:
common_score_abs.describe()

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
count,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000,2370.000000
mean,5.938967,5.375761,4.257100,4.939473,5.461566,5.306570,5.379809,5.382028,5.214452,5.364714,6.535165,8.270042,9.578978,10.419808,11.023180,12.411921,11.702094,11.660275,32.762033,2.558816
std,14.288658,13.267737,11.656376,12.087813,12.497105,12.024250,11.977317,12.034400,11.542797,11.932818,11.632066,11.599794,10.778178,10.813538,10.810750,11.458918,9.909229,9.710542,24.533464,11.549895
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.638298,2.125945,2.957280,4.484143,4.344596,4.462220,11.100448,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.110837,6.979703,7.828341,8.722066,9.857581,9.659819,9.616177,28.662918,0.000000
75%,0.000000,0.000000,0.000000,0.000000,2.605470,3.311789,4.332245,4.393768,4.302041,4.691329,9.420167,12.408532,14.962984,15.361736,15.653885,16.521235,16.273066,16.592293,52.146376,0.000000
max,83.167035,85.662887,86.415671,83.830697,86.157347,90.235591,86.876839,88.228327,85.536586,81.970875,91.059522,86.360487,83.964389,84.547873,83.991328,86.622869,70.420610,70.854900,94.777588,91.279100


In [18]:
summary = common_score_abs.values.flatten()
summary = summary[(summary != 0) & (~pd.isnull(summary))]  # 過濾掉 0 和 NaN

# 計算描述統計
result = pd.Series(summary).describe()
# print(pd.Series(summary).max())
print(result)

count    24097.000000
mean        16.674952
std         15.978122
min          0.001741
25%          5.675558
50%         12.025461
75%         21.639404
max         94.777588
dtype: float64
